<center><div style="direction:rtl;font-family:B Lotus, B Nazanin, Tahoma">به نام خدا</div></center>

<h1><center><div style="direction:rtl;font-family:B Lotus, B Nazanin, Tahoma">رگرسیون یا
Regression داده های ساختار یافته
</div></center></h1>

## <div style="direction:rtl;text-align:right;font-family:B Lotus, B Nazanin, Tahoma">معرفی مجموعه داده تخممین قیمت منازل </div>

<div style="direction:rtl;text-align:right;font-family:Tahoma">
این مجموعه داده در مقاله سال 2016 تحت عنوان 2016
    House price estimation from visual and textual features
معرفی و منتشر شده است.
</div>
<p>
<a href="https://github.com/emanhamed/Houses-dataset" target="_blank">
https://github.com/emanhamed/Houses-dataset
</a>
<br>
<a href="https://arxiv.org/pdf/1609.08399.pdf" target="_blank">
https://arxiv.org/pdf/1609.08399.pdf
</a>
    
</p>

In [1]:
# import the necessary packages
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam
import pandas as pd
import numpy as np
import glob
import cv2
import os
import locale

In [2]:
### آدرس دیتا رو بهش میدیم

cols = ["bedrooms", "bathrooms", "area", "zipcode", "price"]
df = pd.read_csv("https://raw.githubusercontent.com/emanhamed/Houses-dataset/master/Houses%20Dataset/HousesInfo.txt", sep=" ", header=None, names=cols)


In [3]:
df.head()

,bedrooms,bathrooms,area,zipcode,price
0,4,4.0,4053,85255,869500
1,4,3.0,3343,36372,865200
2,3,4.0,3923,85266,889000
3,5,5.0,4022,85262,910000
4,3,4.0,4116,85266,971226


<div style="direction:rtl;text-align:right;font-family:Tahoma">
کد پستی های منحصر به فرد  و تعدادشان
</div>

In [4]:
zipcodes , counts= np.unique(df["zipcode"], return_counts=True)
#dict(zip(zipcodes, counts))

for zipcodes , counts in zip(zipcodes , counts):
  if counts > 20:
      print(zipcodes , counts)
#### دلیل این کارمون اینه که برای مناطقی که کم سمپل داشته باشیم قابلیت پردیکت کردن سخت میشه و نمیشه

91901 32
92276 100
92677 26
92880 49
93446 54
93510 60
94501 41
94531 22


In [5]:
df.shape

(535, 5)

<div style="direction:rtl;text-align:right;font-family:Tahoma">
    حذف رکوردهایی که zip code آن منزل ها zip code شان کمتر از 25 بار تکرا شده است
</div>

In [6]:
print(type(zipcodes), type(counts))  # باید هر دو iterable باشند، مانند list یا tuple


<class 'numpy.int64'> <class 'numpy.int64'>


In [10]:
# loop over each of the unique zip codes and their corresponding
# count
zipcodes = [zipcodes] if isinstance(zipcodes, np.int64) else zipcodes
counts = [counts] if isinstance(counts, np.int64) else counts
for (zipcode, count) in zip(zipcodes, counts):
    #the zip code counts for our housing dataset is *extremely*
    #unbalanced (some only having 1 or 2 houses per zip code)
    # so let's sanitize our data by removing any houses with less
    # than 25 houses per zip code
    if count < 25:
        idxs = df[df["zipcode"] == zipcode].index
        df.drop(idxs, inplace=True)

In [7]:
df["zipcode"]

0      85255
1      36372
2      85266
3      85262
4      85266
       ...  
530    94531
531    94531
532    94531
533    94531
534    94531
Name: zipcode, Length: 535, dtype: int64

In [8]:
# فرض می‌کنیم zipcode و count هر دو از نوع numpy.int64 هستند و فقط شامل یک عدد هستند.
###if counts < 25:
    ###idxs = df[df["zipcode"] == zipcode].index
    ###df.drop(idxs, inplace=True)


NameError: name 'zipcode' is not defined

In [12]:
df.shape

(531, 5)

<div style="direction:rtl;text-align:right;font-family:Tahoma">
جدا کردن داده های موجود به داده های آموزش و آزمون</div>

In [13]:
(train, test) = train_test_split(df, test_size=0.25, random_state=42)
print(train.shape)
print(test.shape)

(398, 5)
(133, 5)


## <div style="direction:rtl;text-align:right;font-family:B Lotus, B Nazanin, Tahoma">پیش پردازش داده ها:</div>

<div style="direction:rtl;text-align:right;font-family:Tahoma">
نرمال سازی داده ها یا Scaling  داده ها بین 0 و 1
</div>

In [14]:
# find the largest house price in the training set and use it to
# scale our house prices to the range [0, 1] (this will lead to
# better training and convergence)
maxPrice = train["price"].max()
trainY = train["price"] / maxPrice
testY = test["price"] / maxPrice

In [15]:
# initialize the column names of the continuous data
continuous = ["bedrooms", "bathrooms", "area"]

# performin min-max scaling each continuous feature column to
# the range [0, 1]
cs = MinMaxScaler()
trainContinuous = cs.fit_transform(train[continuous])
testContinuous = cs.transform(test[continuous])

<div style="direction:rtl;text-align:right;font-family:Tahoma">
تبدیل مقادیر گسسته به one-hot</div>

In [ ]:
# one-hot encode the zip code categorical data (by definition of
# one-hot encoing, all output features are now in the range [0, 1])
zipBinarizer = LabelBinarizer().fit(df["zipcode"])
trainCategorical = zipBinarizer.transform(train["zipcode"])
testCategorical = zipBinarizer.transform(test["zipcode"])

In [16]:
zipBinarizer = LabelBinarizer().fit(train["zipcode"])
trainCategorical = zipBinarizer.transform(train["zipcode"])
testCategorical = zipBinarizer.transform(test["zipcode"])


<div style="direction:rtl;text-align:right;font-family:Tahoma">
    ترکیب ویژگی‌های گستته و پیوسته
</div>

In [17]:
# construct our training and testing data points by concatenating
# the categorical features with the continuous features
trainX = np.hstack([trainCategorical, trainContinuous])
testX = np.hstack([testCategorical, testContinuous])

print(trainX.shape)
print(testX.shape)

(398, 48)
(133, 48)


## <div style="direction:rtl;text-align:right;font-family:B Lotus, B Nazanin, Tahoma">تعریف معماری مدل (model architecture)</div>

In [20]:
dim = trainX.shape[1]
# define our MLP network
model = Sequential()
model.add(Dense(8, input_dim=dim, activation="relu"))
model.add(Dense(4, activation="relu"))
model.add(Dense(1, activation="linear"))

## <div style="direction:rtl;text-align:right;font-family:B Lotus, B Nazanin, Tahoma">Compile model</div>

In [23]:
import tensorflow as tf

In [24]:
opt = tf.keras.optimizers.legacy.Adam(lr=1e-3, decay=1e-3 / 200)
model.compile(loss="mean_absolute_percentage_error", optimizer=opt)

/usr/local/lib/python3.10/dist-packages/keras/src/optimizers/legacy/adam.py:118: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


## <div style="direction:rtl;text-align:right;font-family:B Lotus, B Nazanin, Tahoma">آموش مدل با داده‌های آموزشی</div>

In [25]:
model.fit(trainX, trainY, validation_data=(testX, testY),
    epochs=200, batch_size=8)

Epoch 1/200
50/50 [==============================] - 3s 7ms/step - loss: 140.2897 - val_loss: 69.7118
Epoch 2/200
50/50 [==============================] - 0s 4ms/step - loss: 67.1011 - val_loss: 59.0435
Epoch 3/200
50/50 [==============================] - 0s 4ms/step - loss: 53.5928 - val_loss: 55.9356
Epoch 4/200
50/50 [==============================] - 0s 4ms/step - loss: 46.3029 - val_loss: 55.4449
Epoch 5/200
50/50 [==============================] - 0s 4ms/step - loss: 41.0066 - val_loss: 54.2016
Epoch 6/200
50/50 [==============================] - 0s 4ms/step - loss: 37.9885 - val_loss: 53.5033
Epoch 7/200
50/50 [==============================] - 0s 4ms/step - loss: 36.8645 - val_loss: 54.7391
Epoch 8/200
50/50 [==============================] - 0s 4ms/step - loss: 33.6879 - val_loss: 53.4860
Epoch 9/200
50/50 [==============================] - 0s 5ms/step - loss: 32.3336 - val_loss: 51.5907
Epoch 10/200
50/50 [==============================] - 0s 4ms/step - loss: 31.5928 - val_lo

In [27]:
model.evaluate(trainX)

13/13 [==============================] - 0s 4ms/step - loss: 0.0000e+00


0.0

In [28]:
preds = model.predict(testX)

5/5 [==============================] - 0s 3ms/step


In [35]:
# make predictions on the testing data
preds = model.predict(testX)

# compute the difference between the *predicted* house prices and the
# *actual* house prices, then compute the percentage difference and
# the absolute percentage difference
diff = preds.flatten() - testY
percentDiff = (diff / testY) * 100
absPercentDiff = np.abs(percentDiff)

# compute the mean and standard deviation of the absolute percentage
# difference
mean = np.mean(absPercentDiff)
std = np.std(absPercentDiff)

# finally, show some statistics on our model
locale.setlocale(locale.LC_ALL, "en_US.UTF-8")
print("avg. house price: {}, std house price: {}".format(
    locale.currency(df["price"].mean(), grouping=True),
    locale.currency(df["price"].std(), grouping=True)))
print("mean: {:.2f}%, std: {:.2f}%".format(mean, std))

5/5 [==============================] - 0s 3ms/step
avg. house price: $590,040.23, std house price: $510,817.79
mean: 43.79%, std: 106.05%


<hr>

## <div style="direction:rtl;text-align:right;font-family:B Lotus, B Nazanin, Tahoma">‌منبع</div>   

https://www.pyimagesearch.com/2019/01/21/regression-with-keras/